In [29]:
%%bash

mkdir -p turing-gen-data

if ! [ -f svms-data/url.train.svm ]; then
  cp urls-data/all00.svm turing-gen-data/url.train.svm
fi

if ! [ -f svms-data/url.test.svm ]; then
  cp urls-data/all01.svm turing-gen-data/url.test.svm
fi

In [18]:
ks = [1, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16, 18, 20, 22, 25, 30, 35, 40, 50]
ks_str = ' '.join(map(str, ks))

In [31]:
%%bash -s '{ks_str}'

ks="$1"
echo $ks

cd turing-gen-data/

budget=2048
for k in $ks ; do
for t in train test ; do 
  ln -s url.${t}.svm url.${t}${k}.svm
done
  echo $k
done | tail -2 | RAYON_NUM_THREADS=4 xargs -L1 -I {} -P $(( $(nproc) / 4 )) bash -c "../csl/target/release/csl \
    --budget $budget \
    --force-filter {} --glauber-samples $(( 100 * 1000 * 1000 )) \
    --compress Identity --train url.train{}.svm --max-k 0 \
    --print-new-edges --valid url.test{}.svm > url.k{}.log"

1 2 3 4 5 6 8 10 12 14 16 18 20 22 25 30 35 40 50


In [40]:
%%bash -s '{ks_str}'

ks="$1"
cd turing-gen-data/


if true ; then

function exarr {
key=$1
value=$(grep "^$key" $2 | head -1)
value=${value#"$key"}
echo "\"$3\":" \""$value"\" ","
}

echo "["

for k in 40 50 ; do

logfile="url.k${k}.log"

echo "{"
echo "\"k\": $k,"
echo "\"budget\": 2048,"
#exarr "greedy num colors" $logfile "greedy_colors"
#exarr "filtered thresholded Good-Turing estimate" $logfile "good_turing_new_edge_estimator"
#exarr "filtered thresholded actual new edge avg" $logfile "test_set_new_edges_per_example"
exarr "color collision count" $logfile "greedy_cc"
exarr "color collision std" $logfile "greedy_std"
echo "},"

done
echo "]"
# npm install -g relaxed-json
fi | rjson /dev/stdin > extracted-log.json

In [39]:
%%bash
cd turing-gen-data/
 # todo cd at top of this file
cat extracted-log.json

[
  {
    "k": 40,
    "budget": 2048,
    "greedy_colors": " 1914",
    "good_turing_new_edge_estimator": "",
    "test_set_new_edges_per_example": "",
    "greedy_cc": " 0.052270397126477554",
    "greedy_std": " 0.3058790100862315"
  },
  {
    "k": 50,
    "budget": 2048,
    "greedy_colors": " 1914",
    "good_turing_new_edge_estimator": "",
    "test_set_new_edges_per_example": "",
    "greedy_cc": " 0.057349420384814956",
    "greedy_std": " 0.3422246334118779"
  }
]


In [ ]:
# TODO then awk transform colliding features 